1- criar navegador

2- importar/visualizar a base de dados

3- para cada item dentro da nossa base de dados, para cada produto

    - procurar esse produto no google shopping
        - verficar se algum dos produtos do google shopping está dentro da minha faixa de preço
    - procurar esse produto no buscape
        - verficar se algum dos produtos do buscapé está dentro da minha faixa de preço

4 - salvar as ofertas boas em um dataframe (tabela)

5 - exportar pro excel

6 - enviar por email o resultado da tabela

## Iniciação do Navegador

In [57]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

# Criar navegador
navegador = webdriver.Chrome()

# importar/visualizar a base de dados
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,5000
1,rtx 3060,zota galax,4000,4500
2,cueca lupo,zorba,20,100
3,playstation 5,dualshock,2000,5000
4,tv smart,DLED,1600,6000
5,air fryer,forno,120,600
6,takopi,capa dura,20,60
7,boa noite punpun,capa dura,20,60
8,verão em que hikaru morreu,capa dura,20,60
9,impressora,cartucho,450,700


In [58]:
def verificar_tem_termos_banidos(lista_termos_banidos, nome_produto):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome_produto:
            tem_termos_banidos = True
    return tem_termos_banidos

def verificar_tem_todos_termos_produtos(lista_termos_nome_produto, nome_produto):
    tem_todos_termos_produtos = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome_produto:
            tem_todos_termos_produtos = False

    return tem_todos_termos_produtos

def buscar_mercadolivre(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    # tratar os valores
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    lista_ofertas = []

    # Buscar na amazon
    nav.get("https://www.mercadolivre.com.br/")
    nav.find_element(By.XPATH, '//*[@id="cb1-edit"]').send_keys(produto, Keys.ENTER)
    time.sleep(5)

    # Pegar os resultados
    lista_resultados = nav.find_elements('class name', 'ui-search-layout__item')
    for resultado in lista_resultados:
        nome_produto = resultado.find_element(By.CLASS_NAME, 'poly-component__title-wrapper').text
        nome_produto = nome_produto.lower()
        preco_produto = resultado.find_element(By.CLASS_NAME, 'andes-money-amount__fraction').text
        link = resultado.find_element(By.CLASS_NAME, 'poly-component__title').get_attribute('href')

        # Analisar se o resultado tem termo banido e tem todos os termos do produto
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome_produto)
    
        # Analisar se ele tem TODOS os termos do nome do produto
        tem_todos_termos_produtos = verificar_tem_todos_termos_produtos(lista_termos_nome_produto, nome_produto)
    
        # Analisar se o preco está entre o preco minimo e o preco maximo
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco_produto = preco_produto.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco_produto = float(preco_produto)
            if preco_minimo <= preco_produto <= preco_maximo:
                lista_ofertas.append((nome_produto, preco_produto, link))
    
    # Retornar a lista de ofertas do buscape
    return lista_ofertas
    
    
def buscar_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    # tratar os valores
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    lista_ofertas = []

    # Buscar no buscapé
    nav.get("https://www.buscape.com.br/")
    nav.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)
    
    time.sleep(5)
    # Pegar os resultado
    lista_resultados = nav.find_elements('class name', 'ProductCard_ProductCard_Inner__gapsh')

    for resultado in lista_resultados:
        nome_produto = resultado.find_element(By.TAG_NAME, 'h2').text
        nome_produto = nome_produto.lower()
        preco_produto = resultado.find_element(By.TAG_NAME, 'p').text
        link = resultado.get_attribute('href')

        # Analisar se o resultado tem termo banido e tem todos os termos do produto
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome_produto)
    
        # Analisar se ele tem TODOS os termos do nome do produto
        tem_todos_termos_produtos = verificar_tem_todos_termos_produtos(lista_termos_nome_produto, nome_produto)
    
        # Analisar se o preco está entre o preco minimo e o preco maximo
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco_produto = preco_produto.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco_produto = float(preco_produto)
            if preco_minimo <= preco_produto <= preco_maximo:
                lista_ofertas.append((nome_produto, preco_produto, link))
    
    # Retornar a lista de ofertas do buscape
    return lista_ofertas

## Construindo a nossa tabela de Ofertas

In [59]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    # Pesquisar pelo produto
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]

    lista_ofertas_buscape = buscar_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape: # Condição se a lista tem item
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['Produto', 'Preço', 'Link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else:
        tabela_buscape = None
    
    lista_ofertas_mercado_livre = buscar_mercadolivre(navegador, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_mercado_livre:
        tabela_mercado_livre = pd.DataFrame(lista_ofertas_mercado_livre, columns=['Produto', 'Preço', 'Link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_mercado_livre])
    else:
        tabela_mercado_livre = None
    
display(tabela_ofertas)

,Produto,Preço,Link
0,celular apple iphone 12 64gb 4 gb,4799.00,https://www.buscape.com.br/celular/smartphone-...
0,"apple iphone 12 (64gb) product red 6,1 vermelh...",4338.00,https://produto.mercadolivre.com.br/MLB-542891...
0,gigabyte placa de vídeo geforce rtx 3060 gamin...,4036.57,https://www.buscape.com.br/lead?oid=546338771&...
1,placa de video gigabyte rtx 3060 gaming 12gb,4462.91,https://www.buscape.com.br/lead?oid=1500628760...
0,placa de vídeo nvidia gigabyte eagle geforce r...,4410.00,https://www.mercadolivre.com.br/placa-de-video...
...,...,...,...
3,impressora elgin i7 usb 46i7usbckd00 cor preto,499.00,https://www.mercadolivre.com.br/impressora-elg...
4,impressora multifuncional hp deskjet ink advan...,462.00,https://www.mercadolivre.com.br/impressora-mul...
5,impressora sem ribbon para ecommerce imprime e...,549.00,https://www.mercadolivre.com.br/impressora-sem...
6,impressora função única monocromática control ...,659.00,https://www.mercadolivre.com.br/impressora-fun...


## Exportando para o Excel

In [60]:
# Exporta pro excel
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)

## Enviando o e-mail

In [61]:
# enviar por e-mail o resultado da tabela
import win32com.client as win32

if len(tabela_ofertas) > 0:
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'programacao.oficial2@gmail.com'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f"""
<p>Prezados,</p>
<p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada</p>
{tabela_ofertas.to_html(index=False)}
<p>Att., </p>
"""
    mail.Send()
    navegador.quit()